# TODO (WIP)
This repository is still under development! Still, I wanted to release it before it faded in some remote location of my hard-drive and here it is.  
Keep in mind I **am not** a python developer. Any feedback is welcome.

There is still a lot to do!
- More detailed documentation
- Export of the fields to file to store a current configuration so that it can be retrieved later
- Expose an HTTP interface to generate models on demand
- Integration for simulations?
- Better support for assemblies
- Complete the example model to show all features of the basic class, unlike now :(

# It starts here!

Basic initialization for the notebook, so that cadquery is properly integrated, and the preview on the sidebar is rendered.

In [5]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults

set_defaults(axes=False, grid=True, axes0=True, ortho=True, transparent=True)
set_sidecar("CadQuery", init=True)

Additional libraries specifically needed for this notebook:

In [6]:
import ipywidgets as widgets
import time
import random
import hashlib

## Helper functions
A selection of functions which will be employed across multiple classes and which are nice to have.

In [8]:
# Generate a (almost surely) unique name by combining a pseudo-random hash and a timestamp.
def unique_name():
    return str(int(time.time()))+"_"+hashlib.md5(str(random.randint(0,99999999)).encode('utf-8')).hexdigest()


## LOD class
Support class to describe the level of detail we want for the target model to be generated. It is pretty much the same idea as we would find in most game engines, just for CAD design.  
Ideally we want to use coarser LODs for simplified view or complex assemblies where performance would be affected.  
We want finer LODs for final models which must be exported for machining/printing.

In general we can recognize the following three stages:
- Basic volume: to ensure that this object can fit inside an other. No holes, no fine details like textures or slots. Often it could be implemented as the convex hull of our object. It is meant to be used for cuts.
- Detailed volume: it contains holes and slots, even more so when they have a functional role in the design. It is used for assemblies and fast rendering.
- Complete model: in addition to holes and slots, textures, threads and any other fine detail is captured. They are meant for high quality renderings or final exports to be machined/printed.

More complex and model specific flags can be defined to target a finer control.

In [2]:
class LOD:
    def __init__(self, threads=False, holes=False):
        self.threads = threads
        self.holes = holes

## Model class
Base class which can be used to define all future models. This implementation is arbitrary, and there is plenty of alternative paths which might have been followed.

**Model** is virtual and should not be instanced directly.

In [9]:
class Model:
    _shapes = []
    _holes = []
    _refs  = []
    
    def _build(self, LOD):
        raise Exception("Model is a virtual class")
        return
        
    def _fields(self):
        raise Exception("Model is a virtual class")
        return
    
    def build(self, LOD = LOD()):
        return self._build(LOD)

    def fields(self):
        return self._fields()
    
    # Standard reduction scheme, based on union of shapes minus union of holes.
    # Performance should be profiled to decide which is the best way to implement it.
    def draw(self):
        if len(self._shapes)==0:
            raise Exception("Empty shape list")
            
        tmp = self._shapes[0]
        for v in self._shapes:
            tmp = v.union(tmp)

        for v in self._holes:
            tmp = tmp.cut(v)
            
        return tmp
    
    # Show buttons to control the generation in the viewport and exporting files.
    def gui(self, LOD = LOD()):
        def refresh(model):
            self.build(LOD)
            display(self.draw())
    
        def export_step(model):
            self.build(LOD)
            tmp=self.draw()
            display(tmp)
            tmpFile="./tmp/"+unique_name()+".step"
            cq.exporters.export(tmp, tmpFile)
            link.value="<a href=\""+tmpFile+"\" download>Dowload STEP</a>"
            
        def export_mesh(model):
            self.build(LOD)
            tmp=self.draw()
            display(tmp)
            tmpFile="./tmp/"+unique_name()+".amf"
            cq.exporters.export(tmp, tmpFile)
            link.value="<a href=\""+tmpFile+"\" download>Dowload mesh</a>"
        
        btn_refresh = widgets.Button(description="Refresh")
        btn_export_step = widgets.Button(description="Export STEP")
        btn_export_mesh = widgets.Button(description="Export mesh")
        link = widgets.HTML("<a>[No file]</a>")
        
        btn_refresh.on_click(refresh)
        btn_export_step.on_click(export_step)
        btn_export_mesh.on_click(export_mesh)
        
        display(btn_refresh,btn_export_step,btn_export_mesh,link)
        return
    
    @property
    def shapes(self):
        return self._shape
    
    @shapes.setter
    def shapes(self,val):
        raise Exception("shape is readonly")
        
    @property
    def holes(self):
        return self._holes
    
    @holes.setter
    def holes(self,val):
        raise Exception("holes is readonly")
        
    @property
    def refs(self):
        return self._refs
    
    @refs.setter
    def refs(self,val):
        raise Exception("refs is readonly")

### Example of a derived model
A test class derived from model to show how the full workflow to generate single components is meant.  
In this case it is a simple prism with a square base and a cylindrical hole centered on the vertical axis.

In [10]:
class DerivedModel(Model):
    _par = {"height":0.5}
    def _fields(self):
        #w=widgets.IntSlider()
        #display(w)
        #@widgets.interact(has_hole=True, y=1.0)
        #def g(has_hole, y):
        #    self._shape = [has_hole]
        #    return (has_hole, y)

        #w = widgets.interactive(g, has_hole=True, y=2.0)
        #display(w)
        x = 5
        p = widgets.IntSlider()
        p.value = self._par["height"]
        def on_change(v):
            self._par["height"] = v['new']
        p.observe(on_change, names='value')
        display(p)
        return
    
    def _build(self, LOD):
        self._shapes = [cq.Workplane("front").rect(1,0.5).extrude(self._par["height"])]


We can test this example implementation with few lines of code. Interactive fields and export buttons can be easily generated! 

You can play around with the sliders, and press refresh to see the updated model which can also be exported by generating the temporary file and downloading it.

In [11]:
testModel = DerivedModel()
testModel.build()
display(testModel.draw())

testModel.fields()
testModel.gui()

Done, using side car 'Cadquery'


IntSlider(value=0)

Button(description='Refresh', style=ButtonStyle())

Button(description='Export STEP', style=ButtonStyle())

Button(description='Export mesh', style=ButtonStyle())

HTML(value='<a>[No file]</a>')